Now Applying Gamescore to All Players 

In [3]:
from matplotlib.pyplot import text
import pandas as pd
import re

# Load the main dataframe
df = pd.read_csv("2000-01_pbp.csv")

# Function to extract points from description text
def extract_points(text):
    match = re.search(r'(\d+) PTS', text)
    if match and int(match.group(1)) > 0:
        return int(match.group(1))
    return 0

df['TEXT'] = (
   df['HOMEDESCRIPTION'].fillna('') + ' ' + df['VISITORDESCRIPTION'].fillna('')
)

df['POINTS'] = df['TEXT'].apply(extract_points)

points = df.groupby(['GAME_ID', 'PLAYER1_ID'])['POINTS'].max().reset_index().rename(columns={'PLAYER1_ID': 'PLAYER_ID', 'POINTS ': 'PTS'})


df['TEXT'] = (
    df['HOMEDESCRIPTION'].fillna('') + ' ' +
    df['VISITORDESCRIPTION'].fillna('')
)

df['POINTS'] = df['TEXT'].apply(extract_points)

points = df.groupby(['GAME_ID', 'PLAYER1_ID'])['POINTS'].max().reset_index().rename(columns={'PLAYER1_ID': 'PLAYER_ID', 'POINTS': 'PTS'})

points.sort_values(by='PTS', ascending=False, inplace=True)

#print(points.sort_values(by='PTS', ascending=False).head(10))

reb_df = df[(df['EVENTMSGTYPE'] == 4)]

rebounds = reb_df.groupby(['GAME_ID', 'PLAYER1_ID'])['TEXT'].apply(lambda x: x.str.contains('REBOUND').sum()).reset_index().rename(columns={'PLAYER1_ID': 'PLAYER_ID', 'TEXT': 'REB'})

rebounds.sort_values(by='REB', ascending=False, inplace=True)

#print(rebounds.sort_values(by='REB', ascending=False).head(10))


df['TEXT'] = (
    df['HOMEDESCRIPTION'].fillna('') + ' ' +
    df['VISITORDESCRIPTION'].fillna('')
)

assists_df = df[
    (df['EVENTMSGTYPE'] == 1) &
    (df['PLAYER2_ID'].notna()) &
    (df['PLAYER2_ID'] != 0)
]

assists = (
    assists_df.groupby(['GAME_ID', 'PLAYER2_ID'])
              .size()
              .reset_index(name='AST')
              .rename(columns={'PLAYER2_ID': 'PLAYER_ID'})
)

assists.sort_values('AST', ascending=False, inplace=True)


assists.sort_values(by='AST', ascending=False, inplace=True)

#print(assists.sort_values(by='AST', ascending=False).head(10))

df['TEXT'] = (
    df['HOMEDESCRIPTION'].fillna('') + ' ' +
    df['VISITORDESCRIPTION'].fillna('')
)

steals_df = df[
    (df['EVENTMSGTYPE'] == 5) &
    (df['PLAYER2_ID'].notna())
     & (df['PLAYER2_ID'] != 0)]

steals = (
    steals_df.groupby(['GAME_ID', 'PLAYER2_ID'])
              .size()
              .reset_index(name='STL')
              .rename(columns={'PLAYER2_ID': 'PLAYER_ID'})
)

steals.sort_values(by='STL', ascending=False, inplace=True)

#print(steals.sort_values(by='STL', ascending=False).head(10))


df['TEXT'] = (
    df['HOMEDESCRIPTION'].fillna('') + ' ' +
    df['VISITORDESCRIPTION'].fillna('')
)

blocks_df = df[
    (df['EVENTMSGTYPE'] == 2) &
    (df['PLAYER3_ID'].notna()) &
    (df['PLAYER3_ID'] != 0) &
    (df['TEXT'].str.contains('BLOCK'))
]

blocks = (
    blocks_df.groupby(['GAME_ID', 'PLAYER3_ID'])
              .size()
              .reset_index(name='BLK')
              .rename(columns={'PLAYER3_ID': 'PLAYER_ID'})
)   
blocks.sort_values(by='BLK', ascending=False, inplace=True)

#print(blocks.sort_values(by='BLK', ascending=False).head(10))


Gets the top of each stat for every player, to be used for gamescore

In [4]:
# Merge all stats into a single DataFrame
game_score = points.merge(rebounds, on=['GAME_ID', 'PLAYER_ID'], how='outer') \
    .merge(assists, on=['GAME_ID', 'PLAYER_ID'], how='outer') \
    .merge(steals, on=['GAME_ID', 'PLAYER_ID'], how='outer') \
    .merge(blocks, on=['GAME_ID', 'PLAYER_ID'], how='outer')      

game_score.fillna(0, inplace=True)

# Calculate Game Score using the formula
game_score['GAME_SCORE'] = (
    game_score['PTS'] +
    0.4 * game_score['REB'] + 
    0.7 * game_score['AST'] +
    game_score['STL'] +
    0.7 * game_score['BLK']
)

game_score.sort_values(by='GAME_SCORE', ascending=False, inplace=True)

#add player names
players = pd.read_csv("2000-01_pbp.csv")[['PLAYER1_ID', 'PLAYER1_NAME']].drop_duplicates().rename(columns={'PLAYER1_ID': 'PLAYER_ID', 'PLAYER1_NAME': 'PLAYER_NAME'})
game_score = game_score.merge(players[['PLAYER_ID', 'PLAYER_NAME']], on='PLAYER_ID', how='left')

# Reorder columns to have PLAYER_NAME right after PLAYER_ID
game_score = game_score[['GAME_ID', 'PLAYER_ID', 'PLAYER_NAME', 'PTS', 'REB', 'AST', 'STL', 'BLK', 'GAME_SCORE']]

# Set pandas display options to show all columns on one line
pd.set_option('display.width', 200)
pd.set_option('display.max_columns', None)

print(game_score.head(10))


    GAME_ID  PLAYER_ID       PLAYER_NAME   PTS   REB   AST  STL  BLK  GAME_SCORE
0  20000471        185      Chris Webber  51.0  26.0   5.0  3.0  2.0        69.3
1  20001068        711  Jerry Stackhouse  57.0   4.0   5.0  1.0  0.0        63.1
2  20000557        952    Antoine Walker  47.0   5.0  13.0  4.0  1.0        62.8
3  20000267       1712    Antawn Jamison  51.0  13.0   5.0  2.0  1.0        62.4
4  20000247       1712    Antawn Jamison  51.0  14.0   2.0  3.0  2.0        62.4
5  20000267        977       Kobe Bryant  51.0   7.0   8.0  2.0  1.0        62.1
6  20001140       1503     Tracy McGrady  49.0   8.0   7.0  2.0  4.0        61.9
7  20000477        947     Allen Iverson  54.0   3.0   3.0  3.0  1.0        61.0
8  20000710        950   Stephon Marbury  50.0   3.0  12.0  0.0  0.0        59.6
9  20000138       1713      Vince Carter  48.0  10.0   4.0  3.0  2.0        59.2
